In [1]:
import pandas as pd
import pulp

data = pd.read_csv('data/09202023_all.csv')

In [2]:
import pandas as pd
import pulp


def get_optimal_lineup(data, exclude_players=None):
    """Solve the optimization problem for the given data and constraints."""
    prob = pulp.LpProblem("FantasyFootball", pulp.LpMaximize)
    x = pulp.LpVariable.dicts("Chosen", data.index, cat="Binary")

    # Objective function
    prob += pulp.lpSum(data['projected_points'][i] * x[i] for i in data.index) - 0.1 * pulp.lpSum(data['rank_ecr'][i] * x[i] for i in data.index)

    # Constraints
    prob += pulp.lpSum(data['salary'][i] * x[i] for i in data.index) <= 50000
    prob += pulp.lpSum(x[i] for i in data.index if data.iloc[i]['player_position_id'] == 'QB') == 1
    prob += pulp.lpSum(x[i] for i in data.index if data.iloc[i]['player_position_id'] == 'RB') == 2
    prob += pulp.lpSum(x[i] for i in data.index if data.iloc[i]['player_position_id'] == 'WR') == 3
    prob += pulp.lpSum(x[i] for i in data.index if data.iloc[i]['player_position_id'] == 'TE') == 1
    prob += pulp.lpSum(x[i] for i in data.index if data.iloc[i]['player_position_id'] in ['RB', 'WR', 'TE']) == 3+2+1+1
    prob += pulp.lpSum(x[i] for i in data.index if data.iloc[i]['player_position_id'] == 'DST') == 1

    # Exclude specific players if provided
    if exclude_players:
        for player_idx in exclude_players:
            prob += x[player_idx] == 0

    # Solve and return selected player indices
    prob.solve()
    return [i for i in data.index if x[i].value() == 1]

# Generate 5 diverse optimal lineups
lineups = []
N = 3  # At least N players should be different between lineups

for _ in range(5):
    current_lineup = get_optimal_lineup(data, exclude_players=[player for lineup in lineups for player in lineup[:N]])
    lineups.append(current_lineup)

# Display the selected players along with their rank, projected points, and salary
for idx, lineup in enumerate(lineups, 1):
    total_salary = 0
    print(f"Lineup {idx}:")
    for i in lineup:
        player = data.iloc[i]
        total_salary += player['salary']
        print(f"{player['player_name']} ({player['player_position_id']}): Rank {player['rank_ecr']}, Projected Points {player['projected_points']}, Salary ${player['salary']}")
    print(f"Total Salary for Lineup {idx}: ${total_salary}\n")

Lineup 1:
patrickmahomes (QB): Rank 1, Projected Points 25.5, Salary $8300.0
joemixon (RB): Rank 4, Projected Points 16.3, Salary $6500.0
kennethwalker (RB): Rank 8, Projected Points 15.8, Salary $6200.0
kyrenwilliams (RB): Rank 9, Projected Points 15.6, Salary $5500.0
pukanacua (WR): Rank 13, Projected Points 15.0, Salary $6100.0
deebosamuel (WR): Rank 15, Projected Points 14.5, Salary $5900.0
georgepickens (WR): Rank 24, Projected Points 13.3, Salary $5300.0
traviskelce (TE): Rank 1, Projected Points 19.6, Salary $7200.0
jets (DST): Rank 7, Projected Points 8.1, Salary $2800.0
Total Salary for Lineup 1: $53800.0

Lineup 2:
jalenhurts (QB): Rank 2, Projected Points 23.3, Salary $8000.0
kyrenwilliams (RB): Rank 9, Projected Points 15.6, Salary $5500.0
d'andreswift (RB): Rank 14, Projected Points 13.9, Salary $5700.0
milessanders (RB): Rank 15, Projected Points 13.9, Salary $5700.0
pukanacua (WR): Rank 13, Projected Points 15.0, Salary $6100.0
deebosamuel (WR): Rank 15, Projected Points